In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from functools import partial
import sklearn.datasets
import scipy.linalg
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Matrices como transformaciónes lineales

Sea una base de datos (tabla) con atributos continuos $V \in \mathbb{R}^{M\times D}$
- Cada ejemplo tiene $D$ atributos
- Si asumimos que $0^D$ es el origen entonces cada ejemplo es un **vector** $D$-dimensional 
- Existen $M$ vectores $v_i \in \mathbb{R}^D$  en $V$ (filas)

Consideremos el caso $D=2$

In [ ]:
iris_set = sklearn.datasets.load_iris()
V = iris_set.data[-5:, 2:]
display(V)

In [ ]:
fig, ax1 = plt.subplots(figsize=(6, 4), tight_layout=True)
arrow_args = {'width': 0.05, 'length_includes_head': True, 'alpha': 0.5}
ax1.set_aspect('equal')
ax1.set_xlim([0, 6]); ax1.set_ylim([0, 4]); 
ax1.set_xlabel("Largo del pétalo [cm]")
ax1.set_ylabel("Ancho del pétalo [cm]")

for v in V:
    ax1.arrow(0, 0, v[0], v[1], color='k', **arrow_args)
    ax1.scatter(v[0], v[1], cmap=plt.cm.Set3);

- Podemos encontrar el vector promedio (rojo) y visualizarlo geometrícamente
- Si lo restamos podemos mover el origen de nuestros ejes

In [ ]:
Vmean = np.mean(V, axis=0)
ax1.arrow(0, 0, Vmean[0], Vmean[1], color='r', **arrow_args)
ax1.scatter(Vmean[0], Vmean[1], color='k');

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
arrow_args = {'width': 0.01, 'length_includes_head': True, 'alpha': 0.5}
ax.set_aspect('equal')
ax.set_xlim([-0.5, 0.5]); ax.set_ylim([-0.5, 0.5]); 
ax.set_xlabel("Largo del pétalo [cm] - %0.2f" %(Vmean[0]))
ax.set_ylabel("Ancho del pétalo [cm] - %0.2f" %(Vmean[1]))

V_ = V - Vmean 
for v in V_:
    ax.arrow(0, 0, v[0], v[1], color='k', **arrow_args)
    ax.scatter(v[0], v[1], cmap=plt.cm.Set3);

Consideremos una matriz cuadrada $A = \begin{pmatrix} \alpha & \beta \\ \beta & \alpha \end{pmatrix}$

¿Qué le ocurre a nuestros datos centrados si los multiplicamos por $A$?

In [ ]:
fig, ax2 = plt.subplots(1, figsize=(6, 4), tight_layout=True)

def update_plot(alpha, beta):
    A = [[alpha, beta], [beta, alpha]]    
    display(A)
    ax2.cla(); ax2.set_aspect('equal')
    ax2.set_xlim([-0.5, 0.5]); ax2.set_ylim([-0.5, 0.5]);   
    V_ = V - Vmean 
    for v in V_:
        ax2.arrow(0, 0, v[0], v[1], color='k', **arrow_args)
        ax2.scatter(v[0], v[1], cmap=plt.cm.Set3);
    ax2.set_prop_cycle(None)    
    Vn = np.dot(V_, A)
    for v in Vn:
        ax2.arrow(0, 0, v[0], v[1], color='b', **arrow_args)
        ax2.scatter(v[0], v[1], cmap=plt.cm.Set3);    
    
widgets.interact(update_plot, 
                 alpha=FloatSlider_nice(min=-2, max=2, value=1.), 
                 beta=FloatSlider_nice(min=-2, max=2, value=0.));

### Transformación afín

- Sumando un vector podemos trasladar nuestros datos
- Multiplicando por una matriz cuadrada podemos rotar y escalar nuestros datos
- La combinación de estas operaciones se conoce como [**transformación afín**](https://en.wikipedia.org/wiki/Affine_transformation#Image_transformation)

$$
v = A v' + b
$$

### Bases y transformaciones

> La base de un espacio es un conjunto de vectores tal que todos los elementos del espacio se pueden escribir como una combinación de esos vectores

En el caso bidimensional la base estándar es $\begin{pmatrix} 1 \\ 0 \end{pmatrix}, \begin{pmatrix} 0 \\ 1 \end{pmatrix}$

y un vector cualquiera puede representarse como

$$
\begin{pmatrix} x \\ y \end{pmatrix} = x \begin{pmatrix} 1 \\ 0 \end{pmatrix} + y \begin{pmatrix} 0 \\ 1 \end{pmatrix}
$$

Geometricamente podemos interpretar la base como los **ejes coordenados** de nuestro sistema

### Cambio de coordenadas

Una multiplicación por una matriz podría verse entonces como un "cambio de ejes coordenados"

¿Qué le ocurre a la base estándar si la multiplicamos por $A = \begin{pmatrix} a_1 & a_2 \\ a_3 & a_4 \end{pmatrix}$

¿Cómo se ven los datos en el nuevo espacio?

In [ ]:
fig, ax3 = plt.subplots(1, 2, figsize=(6, 4), tight_layout=True)
world_axis = np.array([[1, 0], [0, 1]])

def update_plot(a1, a2, a3, a4):
    A = [[a1, a2], [a3, a4]]
    new_world_axis = np.dot(A, world_axis)
    print(new_world_axis)
    for ax_ in ax3:
        ax_.cla(); ax_.set_aspect('equal')
        ax_.set_xlim([-1, 1]); ax_.set_ylim([-1, 1]);  
        ax_.spines['right'].set_color('none')
        ax_.spines['top'].set_color('none')
        ax_.spines['bottom'].set_position(('data', 0))
        ax_.spines['left'].set_position(('data', 0))        
        ax_.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
        ax_.yaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
    ax3[1].spines['bottom'].set_color('blue') 
    ax3[1].spines['left'].set_color('green') 
    ax3[0].set_title('Espacio original')
    ax3[1].set_title('Proyección en los nuevos ejes')
    for j, c in enumerate(['b', 'g']):
        ax3[0].arrow(0, 0, new_world_axis[0, j], new_world_axis[1, j], color=c, **arrow_args)
    V_ = V - Vmean 
    for v in V_:
        ax3[0].scatter(v[0], v[1], cmap=plt.cm.Set3);
    Vn = scipy.linalg.solve(A, V_.T).T
    for v in Vn:
        ax3[1].scatter(v[0], v[1], cmap=plt.cm.Set3);  
    
widgets.interact(update_plot, 
                 a1=FloatSlider_nice(min=-1, max=1, value=1.), 
                 a2=FloatSlider_nice(min=-1, max=1, value=0.), 
                 a3=FloatSlider_nice(min=-1, max=1, value=0),
                 a4=FloatSlider_nice(min=-1, max=1, value=1));

Los ejes de la base estándar se escalan y rotan obteniendo un nuevo par de ejes 

La proyección de los vectores originales $\vec v$ a los nuevos ejes es
$$
\begin{align}
\begin{pmatrix} x \\ y \end{pmatrix} &=  A \begin{pmatrix} x' \\ y' \end{pmatrix} \\
A^{-1}\begin{pmatrix} x \\ y \end{pmatrix} &=   \begin{pmatrix} x' \\ y' \end{pmatrix} \\
\end{align}
$$

Por ejemplo si $A = \begin{pmatrix} 0.5 & 0 \\ 0 & 1 \end{pmatrix}$ tendríamos una nueva base donde el eje horizontal es "más corto" y la proyección de $(x, y)$ sería $(2x, y)$


Consideremos ahora una transformación en particular  $A = \begin{pmatrix} 1 & 0.5 \\ 0.5 & 1 \end{pmatrix}$ y un vector cualquiera $v = \rho \begin{pmatrix} \sin(\theta) \\ \cos(\theta) \end{pmatrix}$

In [ ]:
from matplotlib import animation 

fig, ax = plt.subplots(1, figsize=(6, 4), tight_layout=True)
world_axis = np.array([[1, 0], [0, 1]])
A = [[1, 0.5], [0.5, 1]]
new_world_axis = np.dot(A, world_axis)
theta = np.linspace(-np.pi, np.pi, num=100)
rho = 0.75

def update_plot(k):    
    v = rho*np.array([np.sin(theta[k]), np.cos(theta[k])])
    ax.cla(); ax.set_aspect('equal')
    ax.set_xlim([-1.2, 1.2]); ax.set_ylim([-1.2, 1.2]);  
    ax.axis('off')
    ax.set_title(r"$\theta$ = %0.4f" %(theta[k]))
    for j in range(2):
        ax.arrow(0, 0, world_axis[0, j], world_axis[1, j], 
                 linestyle='--', color='k', **arrow_args)
        ax.arrow(0, 0, new_world_axis[0, j], new_world_axis[1, j], 
                 linestyle='--', color='b', **arrow_args)
    for v_,c  in zip([v, np.dot(A, v)], ['k', 'b']):
        ax.scatter(v_[0], v_[1], color=c);
        ax.arrow(0, 0, v_[0], v_[1], color=c, **arrow_args)
    
animation.FuncAnimation(fig, update_plot, frames=len(theta), interval=100, 
                        repeat=True, blit=True)

> Existen algunos vectores que al ser transformados por $A$ mantienen su orientación original

En el ejemplo anterior esto ocurre para $\theta = \pm \pi/4 \approx \pm 0.7854$, es decir $v = \frac{\rho}{\sqrt{2}} \begin{pmatrix}  \pm 1 \\ 1\end{pmatrix}$

Los vectores que tienen ese ángulo **son afectados sólo en su escala**

> Esos vectores se conocen como los *vectores propios* de $A$

¿Cómo podemos encontrar los vectores propios para una matriz $A$ cualquiera?

# Problema de los valores/vectores propios

Sea una matriz cuadrada $A \in \mathbb{R}^{D\times D}$

El siguiente sistema de ecuaciones de $D$ ecuaciones

$$
\begin{split}
A \vec v &= \lambda I \vec v \\
(A - \lambda I) \vec v &= 0,
\end{split}
$$

tiene como resultado 
- $\lambda$, los valores propios de $A$
- $\vec v$ los vectores propios de $A$

La solución no trivial de este problema ($\vec v \neq 0$) se obtiene si $(A - \lambda I)$ es singular, es decir

$$
|A - \lambda I | = 0
$$

que resulta en un polinomio de grado $D$ cuyas raices son $\{\lambda_i\}$, $i=1,2,\ldots, D$

Una vez determinado $\lambda_i$ se pueden usar para despejar $\vec v_i$

### Ejemplo

Para la matriz $A$ del ejemplo, si igualamos su determinante a cero tenemos

$$
(1 - \lambda)^2 - 1/4 = 3/4 - 2\lambda + \lambda^2 = 0
$$

osea $\lambda_1 = 1.5$ y $\lambda_2 = 0.5$. Luego para el primer vector propio tenemos un sistema de ecuaciones

$$
\begin{split}
-0.5v_{11} +0.5v_{12} &= 0 \\
0.5 v_{11} -0.5v_{12} &= 0
\end{split}
$$

osea $v_{11} = v_{12}$ con esto podemos construir un vector normalizado genérico $v_1 = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 \\ 1 \end{pmatrix}$

De forma equivalente para $v_2 = \frac{1}{\sqrt{2}}\begin{pmatrix} -1 \\ 1 \end{pmatrix}$

### Ahora usemos Python

Podemos usar las funciones de [`scipy.linalg`](https://docs.scipy.org/doc/scipy/reference/linalg.html#eigenvalue-problems) `eig()` o `eigvals()` (y sus variantes) para resolver el sistema de ecuaciones

`eig` retorna una tupla con los valores y vectores propios

Para el ejemplo anterior:

In [ ]:
import scipy.linalg

A = np.array([[1., 0.5], [0.5, 1]])
evals, evecs = scipy.linalg.eig(A)
display(evals, evecs)

# Descomposicion en valores propios

Existen [múltiples sistemas en física](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors#Applications) que [ocurren naturalmente](https://hubpages.com/education/What-the-Heck-are-Eigenvalues-and-Eigenvectors) como un problema de valores/vectores propios

Sin embargo la aplicación más amplia para este elemento matemático es la **descomposición en vectores propios**

> Descomponer: Expresar un elemento como una suma de partes de más simples

La descomposición que veremos a continuación usa los **vectores propios** como "las partes simples" (base)


## Principal Component Analysis (PCA)

Es un procedimiento estadístico que busca una **transformación ortogonal** para los datos que logre **maximizar su varianza**

Un conjunto de datos $\{x_i\}$ con $i=1,2,\ldots, M$ y $x_i \in \mathbb{R}^D$

Podemos escribirlo como una matriz $X \in \mathbb{R}^{M\times D}$

Asumiendo que los datos **tienen media cero**, podemos calcular su matriz de correlación como 
$$
C = \frac{1}{M} X^T X
$$

donde $C \in \mathbb{R}^{D\times D}$ y $\bar X$ es la media del conjunto

Llamemos $W \in \mathbb{R}^{D\times D}$ a la matriz de proyección y $X' = X W$ los datos proyectados

- Varianza de $X'$: $\frac{1}{M} X'^T X' = \frac{1}{M} W^T X^T X W = W^T C W$
- Ortogonalidad: $W^T W = I$

Luego el problema de PCA se puede escribir como

$$
\max_W W^T C W \text{ sujeto a } W^T W = I
$$

Si usamos *multiplicadores de Lagrange* para incluir la restricción. Luego derivamos e igualamos a cero

$$
\begin{align}
\frac{d}{dW} W^T C W + \Lambda(I- W^T W) &= 0 \nonumber \\ 
CW - \Lambda W &= 0 \nonumber
\end{align}
$$

donde $\Lambda = \lambda I$ y $\lambda = (\lambda_1, \lambda_2, \ldots, \lambda_D)$

> La transformación de PCA consiste en encontrar los valores y vectores propios de $C$

### Ejemplo

Sean los siguientes datos bidimensionales

In [ ]:
X = np.random.multivariate_normal([0, 0], [[0.5, -0.7], [-0.7, 1]], size=1000)
fig, ax = plt.subplots(1, 2, figsize=(6, 4), tight_layout=True, sharex=True, sharey=True)
ax[0].scatter(X[:, 0], X[:, 1], s=10);
ax[0].set_aspect('equal'); ax[1].set_aspect('equal');

Usaremos PCA para encontrar los ejes coordenados de máxima varianza y graficarlos 

Nota de Python: Conviene usar `eigh` que es una versión de `eig` para matrices simétricas

**Resultado:** 
- El eje rojo acumula un 99.5% de la varianza
- El eje verde es ortogonal al rojo
- Los nuevos ejes están decorrelacionados c/r a los originales

In [ ]:
# Resto la media
X_ = X - np.mean(X, axis=0, keepdims=True)
# Calculo la covarianza
C = np.dot(X_.T, X_)/len(X_)
# Calculo los valores y vectores propios de la covarianza
L, W = scipy.linalg.eigh(C)
for i, c in enumerate(['g', 'r']):
    ax[0].arrow(0, 0, W[i, 0], W[i, 1], color=c, **arrow_args)
    ax[0].text(W[i, 0], W[i, 1], "%0.3f" %(L[i]))
# Los datos proyectados
U = np.dot(X, W)
ax[1].scatter(U[:, 0], U[:, 1])
ax[1].spines['bottom'].set_color('g')
ax[1].spines['left'].set_color('r')

## Reducción de dimensionalidad con PCA

Una aplicación típica de PCA es la reducción de dimensionalidad

Recordemos
- La matriz $W$ tiene las mismas dimensiones que $C$
- Las columnas de $W$ son los vector propios
- Cada vector propio tiene un valor propio asociado

> El valor propio $\lambda_i$ asociado a la columna $i$ de $W$ corresponde a la "cantidad de varianza" de dicha columna

Podemos disminuir la dimensión de nuestros datos si proyectamos usando $\widehat W$ en lugar de $W$ 

$\widehat W$ se obtiene uniendo un subconjunto de las columnas de $W$

En particular nos interesa proyectar a los vectores propios de mayor varianza 

- Si queremos visualizar podemos retener los 2 o 3 componentes principales de mayor varianza
- Si sólo queremos reducir dimensión o decorrelacionar debemos encontrar una dimensión $\hat D < D$ apropiada
    - Un criterio típico es retener los vectores propios que acumulen un 90% de la varianza

### Ejemplo: Iris

Base de datos con **cuatro atributos numéricos** asociados a las características de un conjunto de 150 flores del género Iris separadas en 3 clases

<img src="https://www.math.umd.edu/~petersd/666/html/iris_with_labels.jpg">

Podemos visualizar las relaciones entre las cuatro características

In [ ]:
iris_set = sklearn.datasets.load_iris()
X = iris_set.data
Y = iris_set.target
X -= np.mean(X, axis=0)
display(X.shape)

fig, ax = plt.subplots(3, 3, figsize=(6, 4), tight_layout=True, sharex=True, sharey=True)
for i in range(3):
    for j in range(i, 3):
        for y in range(3):
            ax[i, j].scatter(X[Y==y, i], X[Y==y, j+1], s=10, alpha=0.5)
ax[0 ,0].set_ylabel('Sepal length')
ax[1 ,0].set_ylabel('Sepal width')
ax[2 ,0].set_ylabel('Petal length')
ax[2 ,0].set_xlabel('Sepal width')
ax[2 ,1].set_xlabel('Petal length')
ax[2 ,2].set_xlabel('Petal width');

Para hacer una visualización más concisa podemos 

> usar PCA para reducir la dimensión de los datos de 4 a 2

Además podemos estudiar la contribución de cada atributo a los nuevos ejes

In [ ]:
C = np.dot(X.T, X)/len(X)
display(iris_set.feature_names)
L, W = scipy.linalg.eigh(C)
display("Matriz W")
display(W)
display("Valores propios")
display(L)
display("Porcentage de varianza de cada vector propio")
display(100*L/np.sum(L))

P = np.dot(X, W[:, [3, 2]])

fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True)
for y, name in enumerate(iris_set.target_names):
    ax[0].scatter(P[Y==y, 0], P[Y==y, 1], s=10, label=name)
ax[0].legend(); ax[1].set_aspect('equal')
for ax_ in ax:
    ax_.set_ylabel('PC2'); ax_.set_xlabel('PC3')
ax[1].plot([0, 0], [-1 ,1], 'k--', alpha=0.5)
ax[1].plot([-0.5, 2], [0, 0],  'k--', alpha=0.5)
for i, name in enumerate(iris_set.feature_names):
    ax[1].arrow(0, 0, W[i, 3], W[i, 2], color='b', **arrow_args)
    ax[1].text(W[i, 3], W[i, 2], name)


El vector propio con mayor varianza es PC3 (cuarta columna de W)

En este caso podemos notar como las distintas clases de Iris aparecen en distintos rangos de PC3

### Interpretación de los componentes principales

Podemos estudiar como se conforman los componentes principales para entenderlos mejor

En el ejemplo anterior PC3 es influenciado principalmente por el atributo "largo del pétalo". Luego de forma equivalente y en menor medida por "ancho del pétalo" y "largo del sépalo"

PC2 en cambio está comformado principalmente de los atributos de los sépalos

Este ejercicio es útil para poder darle una interpretación a los componentes principales en nuestro análisis

### Ejercicio: *Eigen-faces* o Rostros principales

Para el set de imágenes de 50x37 píxeles de rostros famosos LFW
- ¿Cúales son los rostros principales (vectores propios)?
- ¿Cúanto podemos reducir en dimensionalidad sin afectar la calidad?

In [ ]:
# Descargar la data, esto demora un poco
import sklearn.datasets
lfw_people = sklearn.datasets.fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw_people['data'].astype('float64')
display(X.shape)

In [ ]:
# Visualizando los 21 primeros ejemplos
fig, ax = plt.subplots(3, 7, figsize=(7, 4))
for i, ax_ in enumerate(ax.ravel()):
    ax_.axis('off')
    ax_.imshow(X[i, :].reshape(50, 37), cmap=plt.cm.Greys_r);

In [ ]:
# Visualizando el rostro promedio
fig, ax = plt.subplots(1, figsize=(5, 2))
ax.axis('off')
ax.imshow(np.mean(X, axis=0).reshape(50, 37), cmap=plt.cm.Greys_r);

In [ ]:
# Restamos la media
X_mean = np.mean(X, axis=0, keepdims=True)
X_center = X - X_mean
# Calculamos C
C = np.dot(X_center.T, X_center)/len(X_center)
display(C.shape)
# Obtenemos los valores y vectores propios
L, W = scipy.linalg.eigh(C)
# Ordenar de L más grande a más pequeño
idx = np.argsort(L)[::-1] 
L = L[idx]
W = W[:, idx]
plt.figure(figsize=(5, 3), tight_layout=True)
plt.plot(np.cumsum(100*L/np.sum(L)))
plt.ylabel('Varianza acumulada');
plt.xlabel('Cantidad de componentes principales');

#### Rostro principales

Los 21 componentes principales de mayor varianza

In [ ]:
fig, ax = plt.subplots(3, 7, figsize=(7, 4.5), tight_layout=True)
for i, ax_ in enumerate(ax.ravel()):
    ax_.axis('off')
    ax_.set_title("{0:0.1e}".format(L[i]))
    ax_.imshow(W[:, i].reshape(50, 37), cmap=plt.cm.Greys_r);

#### Reconstrucción de rostros a partir de los componentes principales

Aumente el valor de $k$ para incluir progresivamente más rostros principales a la reconstrucción

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3))
ax[0].axis('off'); ax[1].axis('off'); ax[2].axis('off')
ax[0].set_title('Original')
ax[1].set_title('Reconstrucción')
ax[2].set_title('Error')

d = 0
ax[0].imshow(X[d, :].reshape(50, 37), cmap=plt.cm.Greys_r)
# Esto calcula los coeficientes de la imagen d
P = np.dot(X_center[d, :], W)
# Esto regenera la imagen cero a partir de sus coeficientes
# Xhat = X_mean + np.dot(W, P)
# np.allclose(X[d, :], Xhat[0, :], rtol=1e-3)

var_acum = 100*np.cumsum(L)/np.sum(L)
# ¿Cuantos coeficientes se necesitan para que X se parezca a Xhat?
def update_plot(k):
    Xhat = X_mean + np.dot(W[:, :k], P[:k])
    ax[1].imshow(Xhat.reshape(50, 37), cmap=plt.cm.Greys_r)
    ax[2].imshow(X[d, :].reshape(50, 37) - Xhat.reshape(50, 37), cmap=plt.cm.RdBu_r, vmin=-255, vmax=255)
    display("Porcentaje de varianza:", 100*np.sum(L[:k])/np.sum(L))
widgets.interact(update_plot, k=SelSlider_nice(options=[1, 2, 5, 10, 20, 50, 100, 250, 500, 
                                                        1000, 1500, 1849]));

## Singular Value Decomposition (SVD)

Tema para otra iteración

https://towardsdatascience.com/pca-and-svd-explained-with-numpy-5d13b0d2a4d8

# Non-negative Matrix Factorization (NMF)

Tema para otra iteración